# Merging priceper (incl geo_services) with postcodes from schools (schools_geo/all_schools_geo_both.csv)


In [2]:
import pandas as pd
from sklearn.linear_model import LinearRegression
import statsmodels.api as sm
import statsmodels.formula.api as smf
import statsmodels
#import janitor

import numpy as np

import sklearn
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix

import matplotlib.pyplot as plt
import seaborn as sns
from statsmodels.graphics.gofplots import ProbPlot
import re

import warnings
warnings.filterwarnings("ignore")

## Merge priceper_addedgeo and schools_geo

In [2]:
priceper_geoservices = pd.read_csv("priceper_housing_data/added_geo (rail_cc)/priceper_railcc.csv")

In [3]:
all_schools_geo_both = pd.read_csv("schools/schools_geo/allschools_geo_both_new.csv")

In [9]:
#all_schools_geo_both = all_schools_geo_both.rename(columns={'id': 'postcode'})

In [4]:
list(priceper_geoservices)

['priceper',
 'year',
 'dateoftransfer',
 'propertytype',
 'duration',
 'price',
 'postcode',
 'lad21cd',
 'transactionid',
 'id',
 'tfarea',
 'numberrooms',
 'classt',
 'CURRENT_ENERGY_EFFICIENCY',
 'POTENTIAL_ENERGY_EFFICIENCY',
 'CONSTRUCTION_AGE_BAND',
 'rail_walk_time',
 'rail_bus_time',
 'cc_walk_time',
 'cc_bus_time']

In [5]:
list(all_schools_geo_both)

['postcode',
 'walk_nearest_sec',
 'nearest_sec_name',
 '2000_good_sec_walk',
 '2001_good_sec_walk',
 '2002_good_sec_walk',
 '2003_good_sec_walk',
 '2004_good_sec_walk',
 '2005_good_sec_walk',
 '2006_good_sec_walk',
 '2007_good_sec_walk',
 '2008_good_sec_walk',
 '2009_good_sec_walk',
 '2010_good_sec_walk',
 '2011_good_sec_walk',
 '2012_good_sec_walk',
 '2013_good_sec_walk',
 '2014_good_sec_walk',
 '2015_good_sec_walk',
 '2016_good_sec_walk',
 '2017_good_sec_walk',
 '2018_good_sec_walk',
 '2019_good_sec_walk',
 '2000_good_sec_bus',
 '2001_good_sec_bus',
 '2002_good_sec_bus',
 '2003_good_sec_bus',
 '2004_good_sec_bus',
 '2005_good_sec_bus',
 '2006_good_sec_bus',
 '2007_good_sec_bus',
 '2008_good_sec_bus',
 '2009_good_sec_bus',
 '2010_good_sec_bus',
 '2011_good_sec_bus',
 '2012_good_sec_bus',
 '2013_good_sec_bus',
 '2014_good_sec_bus',
 '2015_good_sec_bus',
 '2016_good_sec_bus',
 '2017_good_sec_bus',
 '2018_good_sec_bus',
 '2019_good_sec_bus',
 'walk_nearest_pri',
 'nearest_pri_name',
 '2

In [6]:
checktypes = all_schools_geo_both.dtypes
print(checktypes)

postcode               object
walk_nearest_sec        int64
nearest_sec_name       object
2000_good_sec_walk     object
2001_good_sec_walk     object
                       ...   
2017_good_pri_bus      object
2018_good_pri_bus      object
2019_good_pri_bus      object
latitude              float64
longitude             float64
Length: 81, dtype: object


In [7]:
complete_data = priceper_geoservices.merge(all_schools_geo_both[['postcode','walk_nearest_sec', 'nearest_sec_name', 'walk_nearest_pri', 'nearest_pri_name']], on='postcode', how='left')

In [8]:
checktypes = complete_data.dtypes
print(checktypes)

priceper                       float64
year                             int64
dateoftransfer                  object
propertytype                    object
duration                        object
price                          float64
postcode                        object
lad21cd                         object
transactionid                   object
id                               int64
tfarea                         float64
numberrooms                    float64
classt                           int64
CURRENT_ENERGY_EFFICIENCY        int64
POTENTIAL_ENERGY_EFFICIENCY      int64
CONSTRUCTION_AGE_BAND           object
rail_walk_time                  object
rail_bus_time                   object
cc_walk_time                    object
cc_bus_time                     object
walk_nearest_sec                 int64
nearest_sec_name                object
walk_nearest_pri                 int64
nearest_pri_name                object
dtype: object


In [9]:
complete_data['year'] = complete_data['year'].astype(str)

In [10]:
list(complete_data)

['priceper',
 'year',
 'dateoftransfer',
 'propertytype',
 'duration',
 'price',
 'postcode',
 'lad21cd',
 'transactionid',
 'id',
 'tfarea',
 'numberrooms',
 'classt',
 'CURRENT_ENERGY_EFFICIENCY',
 'POTENTIAL_ENERGY_EFFICIENCY',
 'CONSTRUCTION_AGE_BAND',
 'rail_walk_time',
 'rail_bus_time',
 'cc_walk_time',
 'cc_bus_time',
 'walk_nearest_sec',
 'nearest_sec_name',
 'walk_nearest_pri',
 'nearest_pri_name']

In [11]:
final_data = complete_data.copy()

In [12]:
def merge_priceper_years(final_data, all_schools_geo_both):
    for year in range(2000, 2020):            
        # Filter columns that start with the year prefix
        #year_cols = [col for col in all_schools_geo_both.columns if col.startswith(str(year))]
        year_pattern = re.compile(f"^{year}")
        year_cols = [col for col in all_schools_geo_both.columns if year_pattern.match(col)]
        for year_col in year_cols:
            # Merge the datasets based on postcode and year
            final_data = final_data.merge(all_schools_geo_both[['postcode', year_col]], 
                                            left_on=['postcode'], 
                                            right_on=['postcode'], 
                                            how='left')
    
    return final_data

In [13]:
final_data['year'] = final_data['year'].astype(str)
finalised_data = merge_priceper_years(final_data, all_schools_geo_both)

In [14]:
finalised_data.to_csv('final_data.csv', index=False)

## Merge IMD data

In [15]:
complete_data = pd.read_csv('final_data.csv')

In [16]:
imd2004 = pd.read_csv('imd/imd2004eng.csv')

In [17]:
imd2004.head()

,SOA,LA CODE,LA NAME,GOR CODE,GOR NAME,IMD SCORE,RANK OF IMD (where 1 is most deprived),INCOME SCORE,RANK OF INCOME SCORE (where 1 is most deprived),EMPLOYMENT SCORE,...,HEALTH DEPRIVATION AND DISABILITY SCORE,RANK OF HEALTH DEPRIVATION AND DISABILITY SCORE (where 1 is most deprived),EDUCATION SKILLS AND TRAINING SCORE,RANK OF EDUCATION SKILLS AND TRAINING SCORE (where 1 is most deprived),BARRIERS TO HOUSING AND SERVICES SCORE,RANK OF BARRIERS TO HOUSING AND SERVICES SCORE (where 1 is most deprived),CRIME AND DISORDER SCORE,RANK OF CRIME AND DISORDER SCORE (where 1 is most deprived),LIVING ENVIRONMENT SCORE,RANK OF LIVING ENVIRONMENT SCORE (where 1 is most deprived)
0,E01000001,00AA,City of London,H,London,6.52,28325,0.01,32241,0.02,...,-1.55,31317,4.18,28215,28.41,8318,-2.07,32371,32.41,7682
1,E01000002,00AA,City of London,H,London,5.14,29941,0.01,32161,0.02,...,-1.23,29798,3.98,28452,25.74,10599,-2.14,32393,22.65,12324
2,E01000003,00AA,City of London,H,London,17.39,15923,0.09,17414,0.08,...,-0.11,17633,5.73,26351,36.80,3359,-2.15,32397,42.24,4355
3,E01000004,00AA,City of London,H,London,15.77,17376,0.06,22362,0.06,...,-0.33,20663,6.01,26003,37.81,2964,-1.77,32058,46.61,3263
4,E01000005,00AA,City of London,H,London,31.76,7419,0.28,4365,0.16,...,0.52,9124,24.77,10619,31.13,6406,-1.03,28870,28.48,9412


In [18]:
geodata = pd.read_csv('geo/ONSPD_MAY_2023_UK/Data/multi_csv/ONSPD_MAY_2023_UK_BN.csv')
geodata = geodata[['pcds','lsoa01','lsoa11','lsoa21']]

In [19]:
complete_geo = complete_data.merge(geodata[['pcds','lsoa01','lsoa11','lsoa21']], 
                                            left_on=['postcode'], 
                                            right_on=['pcds'], 
                                            how='left')

In [20]:
#left merge left on lad21cd, right on 
complete_04 = complete_geo.merge(imd2004[['SOA','IMD SCORE']], 
                                            left_on=['lsoa01'], 
                                            right_on=['SOA'], 
                                            how='left')

In [21]:
# clean up
complete_04.drop('SOA', axis=1, inplace=True)
complete_04.rename(columns={'IMD SCORE': 'imd_2004'}, inplace=True)

In [22]:
imd2007 = pd.read_csv('imd/imd2007eng.csv')

#left merge left on lad21cd, right on 
complete_07 = complete_04.merge(imd2007[['LSOA','IMD SCORE']], 
                                            left_on=['lsoa01'], 
                                            right_on=['LSOA'], 
                                            how='left')

In [23]:
# clean up
complete_07.drop('LSOA', axis=1, inplace=True)
complete_07.rename(columns={'IMD SCORE': 'imd_2007'}, inplace=True)

In [24]:
imd2010 = pd.read_csv('imd/imd2010eng.csv')

#left merge left on lad21cd, right on 
complete_10 = complete_07.merge(imd2010[['lsoacode','imd_score']], 
                                            left_on=['lsoa01'], 
                                            right_on=['lsoacode'], 
                                            how='left')

In [25]:
# clean up
complete_10.drop('lsoacode', axis=1, inplace=True)
complete_10.rename(columns={'imd_score': 'imd_2010'}, inplace=True)

In [26]:
imd2015 = pd.read_csv('imd/imd2015eng.csv')

#left merge left on lad21cd, right on 
complete_15 = complete_10.merge(imd2015[['LSOA code (2011)','Index of Multiple Deprivation (IMD) Score']], 
                                            left_on=['lsoa01'], 
                                            right_on=['LSOA code (2011)'], 
                                            how='left')

In [27]:
# clean up
complete_15.drop('LSOA code (2011)', axis=1, inplace=True)
complete_15.rename(columns={'Index of Multiple Deprivation (IMD) Score': 'imd_2015'}, inplace=True)

In [28]:
#CDRC Harmonised IMD 2019
imd2019 = pd.read_csv('imd/uk_imd2019.csv')
##note its rank and SOA_decile

#left merge left on lad21cd, right on 
complete_19 = complete_15.merge(imd2019[['LSOA','Rank','SOA_decile']], 
                                            left_on=['lsoa11'], 
                                            right_on=['LSOA'], 
                                            how='left')

In [29]:
# clean up
complete_19.drop('LSOA', axis=1, inplace=True)
complete_19.rename(columns={'Rank': 'imd_2019_rank'}, inplace=True)
complete_19.rename(columns={'SOA_decile': 'imd_2019_dec'}, inplace=True)

In [30]:
####complete_19.to_csv('final_imd_data.csv')

## Add Catchment Area variable (sec schools)

In [2]:
main = pd.read_csv('final_imd_data.csv')

In [3]:
catchment = pd.read_csv('schools/BrightonSecondaryCatchments.csv')

In [6]:
main.head()

,priceper,year,dateoftransfer,propertytype,duration,price,postcode,lad21cd,transactionid,id,...,pcds,lsoa01,lsoa11,lsoa21,imd_2004,imd_2007,imd_2010,imd_2015,imd_2019_rank,imd_2019_dec
0,4244.730679,2015,13/11/2015,F,L,290000,BN1 1AD,E06000043,{288DCE29-0585-E510-E050-A8C06205480E},3268426,...,BN1 1AD,E01016952,E01016952,E01016952,41.72,41.25,45.87,31.608,8856,3
1,2927.400468,2003,21/2/2003,F,L,200000,BN1 1AD,E06000043,{8F424A5C-E6DD-492F-8827-7AF9523BB665},3268426,...,BN1 1AD,E01016952,E01016952,E01016952,41.72,41.25,45.87,31.608,8856,3
2,3496.503497,2013,4/12/2013,F,L,500000,BN1 1AD,E06000043,{AECCFEF8-D159-4688-B479-62C68F590D73},3279748,...,BN1 1AD,E01016952,E01016952,E01016952,41.72,41.25,45.87,31.608,8856,3
3,3220.140515,2006,2/11/2006,F,L,220000,BN1 1AD,E06000043,{4863D52D-C206-4176-9706-A19CDFA1678E},3268426,...,BN1 1AD,E01016952,E01016952,E01016952,41.72,41.25,45.87,31.608,8856,3
4,1816.083916,2004,26/8/2004,F,L,259700,BN1 1AD,E06000043,{DEC7C004-0B15-42DA-8620-A4D8B25855E5},3279748,...,BN1 1AD,E01016952,E01016952,E01016952,41.72,41.25,45.87,31.608,8856,3


In [5]:
catchment.head()

,postcode,AreaName
0,BN1 1AD,StringerVarndean
1,BN1 1AJ,StringerVarndean
2,BN1 1AL,StringerVarndean
3,BN1 1AN,StringerVarndean
4,BN1 1AP,StringerVarndean


In [7]:
merged = main.merge(catchment[['postcode','AreaName']], on='postcode', how='left')
merged.head(20)

,priceper,year,dateoftransfer,propertytype,duration,price,postcode,lad21cd,transactionid,id,...,lsoa01,lsoa11,lsoa21,imd_2004,imd_2007,imd_2010,imd_2015,imd_2019_rank,imd_2019_dec,AreaName
0,4244.730679,2015,13/11/2015,F,L,290000,BN1 1AD,E06000043,{288DCE29-0585-E510-E050-A8C06205480E},3268426,...,E01016952,E01016952,E01016952,41.72,41.25,45.87,31.608,8856,3,StringerVarndean
1,2927.400468,2003,21/2/2003,F,L,200000,BN1 1AD,E06000043,{8F424A5C-E6DD-492F-8827-7AF9523BB665},3268426,...,E01016952,E01016952,E01016952,41.72,41.25,45.87,31.608,8856,3,StringerVarndean
2,3496.503497,2013,4/12/2013,F,L,500000,BN1 1AD,E06000043,{AECCFEF8-D159-4688-B479-62C68F590D73},3279748,...,E01016952,E01016952,E01016952,41.72,41.25,45.87,31.608,8856,3,StringerVarndean
3,3220.140515,2006,2/11/2006,F,L,220000,BN1 1AD,E06000043,{4863D52D-C206-4176-9706-A19CDFA1678E},3268426,...,E01016952,E01016952,E01016952,41.72,41.25,45.87,31.608,8856,3,StringerVarndean
4,1816.083916,2004,26/8/2004,F,L,259700,BN1 1AD,E06000043,{DEC7C004-0B15-42DA-8620-A4D8B25855E5},3279748,...,E01016952,E01016952,E01016952,41.72,41.25,45.87,31.608,8856,3,StringerVarndean
5,3146.853147,2017,31/3/2017,F,L,450000,BN1 1AD,E06000043,{4E95D757-551D-EDA1-E050-A8C0630539E2},3328370,...,E01016952,E01016952,E01016952,41.72,41.25,45.87,31.608,8856,3,StringerVarndean
6,2394.636015,2002,19/7/2002,F,L,200000,BN1 1AD,E06000043,{AA558031-5230-4098-9C67-9BA2F051739F},3238110,...,E01016952,E01016952,E01016952,41.72,41.25,45.87,31.608,8856,3,StringerVarndean
7,1188.811189,2003,30/5/2003,F,L,170000,BN1 1AD,E06000043,{102764AC-2FF7-41DD-98E3-59A934A8DC7C},3279748,...,E01016952,E01016952,E01016952,41.72,41.25,45.87,31.608,8856,3,StringerVarndean
8,1538.461538,2002,29/5/2002,F,L,220000,BN1 1AD,E06000043,{48ECAC4C-2469-4DD6-A4AC-32035C19D301},3328370,...,E01016952,E01016952,E01016952,41.72,41.25,45.87,31.608,8856,3,StringerVarndean
9,4533.333333,2021,31/3/2021,F,L,340000,BN1 1AD,E06000043,{C6209F5F-2769-295E-E053-6C04A8C0DDCC},3276403,...,E01016952,E01016952,E01016952,41.72,41.25,45.87,31.608,8856,3,StringerVarndean


In [10]:
merged.rename(columns={'AreaName': 'SecCatchment'}, inplace=True)

In [11]:
merged.to_csv('final_imd_catch.csv', index=False)

## Add Electoral Ward


In [11]:
main = pd.read_csv('2000_cleaned_data_final_oac11.csv')

In [12]:
wards = pd.read_csv('schools/BrightonElectoralWards.csv')

In [13]:
main.head()

,postcode,rowID,priceper,year,dateoftransfer,propertytype,duration,price,lad21cd,transactionid,...,pcds,lsoa01,lsoa11,lsoa21,X2019_imd_rank,X2019_imd_decile,oac11,geometry,X,floorper
0,BN1 1AD,6,4244.730679,2015,13/11/2015,F,L,290000,E06000043,{288DCE29-0585-E510-E050-A8C06205480E},...,BN1 1AD,E01016952,E01016952,E01016952,8856,3,2,c(-0.142167,50.821691),22.773333
1,BN1 1AD,7,2927.400468,2003,21/2/2003,F,L,200000,E06000043,{8F424A5C-E6DD-492F-8827-7AF9523BB665},...,BN1 1AD,E01016952,E01016952,E01016952,8856,3,2,c(-0.142167,50.821691),22.773333
2,BN1 1AD,8,3496.503497,2013,4/12/2013,F,L,500000,E06000043,{AECCFEF8-D159-4688-B479-62C68F590D73},...,BN1 1AD,E01016952,E01016952,E01016952,8856,3,2,c(-0.142167,50.821691),35.750000
3,BN1 1AD,9,3220.140515,2006,2/11/2006,F,L,220000,E06000043,{4863D52D-C206-4176-9706-A19CDFA1678E},...,BN1 1AD,E01016952,E01016952,E01016952,8856,3,2,c(-0.142167,50.821691),22.773333
4,BN1 1AD,10,1816.083916,2004,26/8/2004,F,L,259700,E06000043,{DEC7C004-0B15-42DA-8620-A4D8B25855E5},...,BN1 1AD,E01016952,E01016952,E01016952,8856,3,2,c(-0.142167,50.821691),35.750000


In [14]:
wards.head()

,postcode,WD22CD,WD22NM,WD22NMW,LAD22CD,LAD22NM,BNG_E,BNG_N,LONG,LAT,Shape_Leng,OBJECTID,GlobalID
0,BN1 1AD,E05002432,Regency,,E06000043,Brighton and Hove,530633,104382,-0.14665,50.8242,4786.541407,790,0b3bd8c8-19cb-4b70-852e-3c4da241a953
1,BN1 1AJ,E05002432,Regency,,E06000043,Brighton and Hove,530633,104382,-0.14665,50.8242,4786.541407,790,0b3bd8c8-19cb-4b70-852e-3c4da241a953
2,BN1 1AL,E05002432,Regency,,E06000043,Brighton and Hove,530633,104382,-0.14665,50.8242,4786.541407,790,0b3bd8c8-19cb-4b70-852e-3c4da241a953
3,BN1 1AN,E05002432,Regency,,E06000043,Brighton and Hove,530633,104382,-0.14665,50.8242,4786.541407,790,0b3bd8c8-19cb-4b70-852e-3c4da241a953
4,BN1 1AP,E05002432,Regency,,E06000043,Brighton and Hove,530633,104382,-0.14665,50.8242,4786.541407,790,0b3bd8c8-19cb-4b70-852e-3c4da241a953


In [15]:
print(wards["WD22NM"].unique())

['Regency' "St. Peter's and North Laine" "Queen's Park"
 'Brunswick and Adelaide' 'Goldsmid' 'Preston Park' 'Withdean'
 'Hollingdean and Stanmer' 'Patcham' 'Moulsecoomb and Bevendean'
 'East Brighton' 'Hanover and Elm Grove' 'Rottingdean Coastal'
 'Woodingdean' 'Central Hove' 'Westbourne' 'Wish' 'Hove Park'
 'Hangleton and Knoll' 'South Portslade' 'North Portslade']


In [16]:
print(wards["WD22NM"].value_counts())

WD22NM
Rottingdean Coastal            271
St. Peter's and North Laine    266
Withdean                       240
Goldsmid                       228
Queen's Park                   219
Patcham                        217
Hangleton and Knoll            214
East Brighton                  205
Hove Park                      204
Preston Park                   172
Hanover and Elm Grove          164
Regency                        162
Hollingdean and Stanmer        161
Moulsecoomb and Bevendean      159
South Portslade                158
Central Hove                   157
Wish                           153
Westbourne                     147
Woodingdean                    141
Brunswick and Adelaide         134
North Portslade                130
Name: count, dtype: int64


In [19]:
print(wards["WD22NM"].isna().sum())

0


In [20]:
merged = main.merge(wards[['postcode','WD22NM']], on='postcode', how='left')
merged.head(20)

,postcode,rowID,priceper,year,dateoftransfer,propertytype,duration,price,lad21cd,transactionid,...,lsoa01,lsoa11,lsoa21,X2019_imd_rank,X2019_imd_decile,oac11,geometry,X,floorper,WD22NM
0,BN1 1AD,6,4244.730679,2015,13/11/2015,F,L,290000,E06000043,{288DCE29-0585-E510-E050-A8C06205480E},...,E01016952,E01016952,E01016952,8856,3,2,c(-0.142167,50.821691),22.773333,Regency
1,BN1 1AD,7,2927.400468,2003,21/2/2003,F,L,200000,E06000043,{8F424A5C-E6DD-492F-8827-7AF9523BB665},...,E01016952,E01016952,E01016952,8856,3,2,c(-0.142167,50.821691),22.773333,Regency
2,BN1 1AD,8,3496.503497,2013,4/12/2013,F,L,500000,E06000043,{AECCFEF8-D159-4688-B479-62C68F590D73},...,E01016952,E01016952,E01016952,8856,3,2,c(-0.142167,50.821691),35.750000,Regency
3,BN1 1AD,9,3220.140515,2006,2/11/2006,F,L,220000,E06000043,{4863D52D-C206-4176-9706-A19CDFA1678E},...,E01016952,E01016952,E01016952,8856,3,2,c(-0.142167,50.821691),22.773333,Regency
4,BN1 1AD,10,1816.083916,2004,26/8/2004,F,L,259700,E06000043,{DEC7C004-0B15-42DA-8620-A4D8B25855E5},...,E01016952,E01016952,E01016952,8856,3,2,c(-0.142167,50.821691),35.750000,Regency
5,BN1 1AD,11,3146.853147,2017,31/3/2017,F,L,450000,E06000043,{4E95D757-551D-EDA1-E050-A8C0630539E2},...,E01016952,E01016952,E01016952,8856,3,2,c(-0.142167,50.821691),28.600000,Regency
6,BN1 1AD,12,2394.636015,2002,19/7/2002,F,L,200000,E06000043,{AA558031-5230-4098-9C67-9BA2F051739F},...,E01016952,E01016952,E01016952,8856,3,2,c(-0.142167,50.821691),27.840000,Regency
7,BN1 1AD,13,1188.811189,2003,30/5/2003,F,L,170000,E06000043,{102764AC-2FF7-41DD-98E3-59A934A8DC7C},...,E01016952,E01016952,E01016952,8856,3,2,c(-0.142167,50.821691),35.750000,Regency
8,BN1 1AD,14,1538.461538,2002,29/5/2002,F,L,220000,E06000043,{48ECAC4C-2469-4DD6-A4AC-32035C19D301},...,E01016952,E01016952,E01016952,8856,3,2,c(-0.142167,50.821691),28.600000,Regency
9,BN1 1AD,16,3891.283525,2008,22/8/2008,F,L,325000,E06000043,{8E01FCEF-A751-449F-9388-639432AECDD2},...,E01016952,E01016952,E01016952,8856,3,2,c(-0.142167,50.821691),27.840000,Regency


In [21]:
print(merged["WD22NM"].value_counts())

WD22NM
Goldsmid                       6738
St. Peter's and North Laine    5679
Rottingdean Coastal            5441
Preston Park                   5020
Withdean                       5007
Hanover and Elm Grove          4953
Queen's Park                   4581
Central Hove                   4441
Brunswick and Adelaide         4287
Westbourne                     3841
Regency                        3617
Patcham                        3421
Moulsecoomb and Bevendean      3263
Hangleton and Knoll            3068
Wish                           3001
East Brighton                  2870
Hove Park                      2717
North Portslade                2563
South Portslade                2537
Hollingdean and Stanmer        2486
Woodingdean                    2121
Name: count, dtype: int64


In [22]:
merged.rename(columns={'WD22NM': 'ElectWards'}, inplace=True)

In [23]:
merged.to_csv('2000_cleaned_data_final_oac11_elec.csv', index=False)